[Table of Contents](http://nbviewer.ipython.org/github/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/table_of_contents.ipynb)

# 前言

\addcontentsline{toc}{chapter}{Preface}

In [1]:
#format the book
from book_format import load_style
load_style(name='kf_book/custom_chinese.css') 

卡尔曼与贝叶斯过滤器的入门书籍。本书使用Jupyter的notebook来写作，因此你可以在浏览器中阅读本书并且运行和修改书中的代码，然后观察执行结果。还有什么比这更好的学习方式呢？

<img src="https://raw.githubusercontent.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/master/animations/05_dog_track.gif">


## 卡尔曼与贝叶斯过滤器
我们想要跟踪和测量环境中的事件和数据，但是传感器会有噪声，因此我们不能完全依靠传感器给我们准确的信息。比如，汽车上的GPS设备会测量海拔，但每次我通过同一路段的相同位置的时候，它总会给我略微不同的海拔值。电子秤在两次称量相同物体的过程也会显示不同的读数。

在简单情况的下的解决方案是显而易见的。如果电子秤每次都显示略微不同的值，我们可以多次测量然后取平均值，或者可以使用精确度更高的电子秤。但是如果传感器的噪声非常大，亦或者在环境中多次测量非常困难？比如我们尝试跟踪低空飞行器的移动，或者想要制造自动飞行的无人机，亦或者需要确保农用机在这个麦田上播种。在计算机视觉领域，需要能够跟踪图像中移动的物体，但是计算机视觉算法会有产生很大的噪声以及不可靠的结果。

本书教你如何解决这些过滤问题，也会使用许多不同的算法，但是他们都是基于贝叶斯概率。简单来说，贝叶斯概率根据历史信息来决定事件为真的概率。

如果我问你此时车辆的转角是多少，你可能没有任何概念，你只能从1$^\circ$到360$^\circ$选一个角度，而你有$\frac{1}{360}$的概率能够回答对。现在假设我告诉你2秒前它的转角为$243^\circ$，2秒内我的车不会转得非常快，因此你可以做出更佳准确的预测，利用历史信息来更佳准确的推断现在以及未来。

环境也有噪声，通过预测我们能够更好得做出估计，但也会受限于噪声。我会因为路上的小狗突然刹车，或者为避开坑洼而突然转向。强风或者路面上的积雪都是影响我的汽车形势路径的外部因素，在控制论领域称这些因素为噪声。

贝叶斯概率远不止这些内容，但是你已经理解其主要思想。知识也有不确定性，我们会根据观察到的证据的强弱来调整我们对事件的置信度。卡尔曼于贝叶斯过滤器基于有限的关于系统如何运作的知识以及传感器精度，来计算出可能性最高的系统状态。

假设我们想要跟踪物体，其传感器会报告它的突然转向。它是突然转向还是传感器的数据噪声？这取决于具体的情况，如果是喷气式飞机，会认为该数据是演习操作产生的；如果是沿着铁路轨道的运货火车，就会直接丢掉该数据，并且改变对传感器精度的置信度；对事件的置信度取决于历史信息和被跟踪系统的知识，以及传感器的特性等。

Rudolf Emil Kálmán发明卡尔曼过滤器，采用数学方式来解决该系列问题。它最初应用在阿波罗登月飞行器，然后就开始在不同领域应用。卡尔曼过滤器被用在飞行器、潜艇、巡航导弹、机器人、IoT传感器以及实验室器械。华尔街利用它来跟踪经融市场，化学专家用它来控制和监控化学反应，也被用于医学影像和去除心电图中的噪声。如果是传感器或者时间序列数据，通常都会涉及到卡尔曼或者相关过滤器。

## 本书的目的
我是在航空领域工作了近二十年的软件工程师，经常会同卡尔曼过滤器打交道，但是我从来没有自己去实现过。卡尔曼过滤器以难以理解闻名，虽然其理论非常漂亮，如果你没有信息处理、控制论、概率与统计以及导航与控制理论的基础，会很难理解卡尔曼过滤器。当我开始使用计算机视觉技术来解决跟踪问题的时候，急迫的需要我自己来实现卡尔曼过滤器。

在这个领域已经有许多经典的教科书，比如Andrew的《Kalman Filtering》。如果你没有特定的背景知识，当你坐下来尝试阅读这些书籍是非常沮丧😦和痛苦😖过程。通常这些书籍的前几章都是快速的回顾大学的高等数学知识，并且会把你引向微积分的教科书，然后用几个段落来总结整个学期的统计知识。这些书籍都是高年级大学生或者研究生的课程，也是研究人员或者教授的参考书籍，但是对于自学的人来说实在是太难了。书中的符号也未加任何解释，不同的教科书使用不同的词语和变量名描述相同的概念，这些书籍也缺乏例子以及恰当的问题。我经常发现我自己能够分析这些词语以及理解数学定义，但是没有任何概念如果运用在真是的现象上面或者这些数学真正想要尝试描述的内容。“这是都是些什么意思呢？”，这是经常我问自己的问题。下面就是一个曾经困扰我的经典例子：

$$\begin{aligned}\hat{x}_{k} = \Phi_{k}\hat{x}_{k-1} + G_k u_{k-1} + K_k [z_k - H \Phi_{k} \hat{x}_{k-1} - H G_k u_{k-1}]
\\ 
\mathbf{P}_{k\mid k} = (I - \mathbf{K}_k \mathbf{H}_{k})\textrm{cov}(\mathbf{x}_k - \hat{\mathbf{x}}_{k\mid k-1})(I - \mathbf{K}_k \mathbf{H}_{k})^{\text{T}}  + \mathbf{K}_k\textrm{cov}(\mathbf{v}_k )\mathbf{K}_k^{\text{T}}\end{aligned}$$

然而当我开始真正理解卡尔曼过滤器的时候，我意识到其基本概念确实非常的直观简单。如果你稍稍有简单概率论的知识，以及一些不确定性知识的直觉，卡尔曼过滤器的概念非常容易理解。卡尔曼过滤器以难以理解闻名，但是去掉那些专业术语之后，卡尔曼过滤器的优美以及背后的数学知识变得更加清晰了，而我也爱上了这个主题。

当我开始理解数学和理论后，更加困难问题出现在我面前。书籍或者文献资料会陈述某些事实并且给出图形作为证明。但是不幸的是，为何这些陈述对我而言却不是那么容易理解，或者我无法重现这些绘图。或者我会想“R=0的时候为真？”，或者作者提供的非常高层次的伪代码在实现的时候就那么显而易见了。某些书籍也提供了Matlab代码，但是我没法购买这些昂贵软件的许可。这些书籍的每个章节都有许多有用的练习题，如果你想要自己去实现卡尔曼过滤器，就需要去理解这些练习题，但是通常这些练习题通常没有答案。如果在课堂上选择使用这些书籍，将会是不错的选择，但是对于自学的人来说却比较糟糕。我讨厌作者对我隐瞒练习题答案，但是作者的本意大概是为了避免在课堂上的学生作弊吧！

上述的这些都阻碍了学习。当我想要追踪屏幕上的图像，或者给我的Arduino项目写代码的时候，我想知道如何实现书中的图形，以及选择不同的参数;在运行仿真模拟的时候，想在信号中加入更多的噪声然后观察过滤器的效果。火箭科学家和研究人员每天都有许多的机会用到卡尔曼过滤器。

我写这本书就是为了满足这些需求。如果你想要设计军用雷达，本书或许不是你最好的选择，建议去STEM学校拿硕士或者PhD学位，因为你需要更多知识才能实现这些东西。本书主要面向业余爱好者以及需要做数据过滤和平滑的工程师们。如果你是业余爱好者，那么本书会提供你所需的一切。如果还想深入研究卡尔曼过滤器，你还需要学习其他的知识。我的目的是介绍足够的概念和数学知识，以便你更好的理解教科书和文献资料。

本书是可交互式的，虽然你可以在线阅读静态内容，我强烈建议你通过交互式的方式来学习和使用它。使用Jupyter的Notebook书写，允许我将文本、数学、Python代码以及Python输出放在一起。如果想要改变参数的值？直接notebook中改变参数的值，按CTRL-ENTER来观察新的图形或者输出。

本书也有练习题，以及答案。我相信你，如果你只是需要一个答案，那就直接阅读答案就行了。如果你想要消化这些新知识，那就在阅读答案前尝试实现练习题的内容。因为本书是交互式的，你不需要不同的环境，或者在开始前引入各种依赖文件。

本书同时也是开源免费的，我花费了很多钱在卡尔曼过滤器的书籍上，我不相信在经济落后的地方或者经济拮据的学生能够买得起那些昂贵的书籍。开源软件让我获益良多，比如Python和Allen B的免费书籍。因此，本书也是免费的，托管在Github上，并且本书只使用开源软件，比如IPython和MathJax。

## 在线阅读
<b>Github</b>

本书托管在Github上，你可以阅读任何章节。Github会静态的渲染Jupyter Notebook，虽然你不能直接运行和修改代码，但是你可以阅读所有内容。

Github英文项目地址：

https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python 
    
GIthub中文翻译地址：

https://github.com/deyuhua/Kalman-and-Bayesian-Filters-in-Python
    
<b>binder</b>

binder提供在线的交互式的notebook访问，因此你可以直接在浏览器中运行和修改代码，不需要下载本书籍或者安装Jupyter。

http://mybinder.org/repo/rlabbe/Kalman-and-Bayesian-Filters-in-Python
    
<b>nbviewer</b>

nbviewer会静态渲染Jupyter Notebook，我发现它比Github渲染的更好，但是也会稍稍更难使用。

http://nbviewer.ipython.org/github/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/table_of_contents.ipynb

## PDF版本
我会定期从notebook生成pdf文件，你可以通过下面的地址访问：

https://drive.google.com/file/d/0By_SW19c1BfhSVFzNHc0SjduNzg/view?usp=sharing

## 下载书籍
本书是交互式的，我强烈推荐你采用这种方式使用它。如果你在你的电脑上安装了IPython，并且克隆了本书，那你就能够在浏览器中运行书中的所有代码。你可以自己实验，观察不同的过滤器如何应用于不同的数据，或者不同的过滤器应用于相同的数据等等。我发现这种立即反馈是必不可少的，你不必去猜测“如果这样会发生什么？”，最好的方式就是尝试然后观察结果！

Github英文主页：
    
https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python
    
GIthub中文翻译主页：
    
https://github.com/deyuhua/Kalman-and-Bayesian-Filters-in-Python
    
你可以使用下面的命令克隆到你的本地

git clone --depth=1 https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python.git

该命令会在本地文件系统创建名为Kalman-and-Bayesian-Filters-in-Python的文件夹。除非你需要查看我真个提交的历史，添加depth参数是为了获取最新的版本，这样速度会更快并且非常节约空间。切换到该目录，使用下面的命令运行Jupyter Notebook：

    jupyter notebook
    
该命令会打开浏览器的窗口，显示主文件夹中的内容。本书是分章节的，每章节都是命令为xx-name.ipynb，xx表示章节数。

## 安装与软件要求
需要安装的软件都在附录中有详细的描述，你可以在线阅读：
    
http://nbviewer.ipython.org/github/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/Appendix_A_Installation.ipynb

你需要安装Ipython3.0或者更高的版本，Python 2.7+ or Python 3.4+, NumPy, SciPy, SymPy, 以及Matplotlib。也需要我的开源软件包FilterPy。

最简单的方式安装这些依赖软件就是安装免费科学Python包，比如Anaconda https://store.continuum.io/cshop/anaconda/ 。

安装FilterPy：
    
    pip install filterpy

## 我的软件库和模块
我也编写了开源软件贝叶斯过滤器的Python库FilterPy，安装方法已经在上面给出。

FilterPy托管在Github(https://github.com/rlabbe/filterpy) ,但是可以通过pip直接安装。

本书的代码都保存在子文件夹code下，包含用于格式化本书以及名为xxx_internal.py的文件。我把特定章节下部分的代码保存在这些函数中，目的隐藏哪些不重要也无需阅读的代码，比如我画图表的代码，或者我想让你更加关注图的内容而非我用Python生成这些图的原理的时候。如果你对这些背后的原理也感兴趣，直接阅读相关代码就行。

在某些章节介绍的函数在后续章节也用到，这些函数除了定义在Notebook中，还将代码保存在code文件夹下，以便后面的章节能够快速引入。但是我尝试避免这种情况，因为我经常遇到代码同步的问题。这是现在知道的唯一的方式在不同notebook之间共享代码，Jupyter没有提供引用其他notebook中的代码单元的方法。

experiments文件夹主要保存我写的测试代码，但也有许多有趣的内容，有兴趣的话也可以去阅读。随着书籍逐步完善，我计划添加更多实例和项目。有些小的实验内容最终会被删除，如果你只是想阅读书籍，完全可以忽略该文件夹。

coda文件保存了本书的样式文件，Jupyter默认的风格和样式过于简朴。本书的风格基本是参照其他类似的书籍《[Probabilistic Programming and Bayesian Methods for Hackers](http://nbviewer.ipython.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Chapter1.ipynb) 》[2]，我也深受Lorena Barba教授的影响，[相关资料](https://github.com/barbagroup/CFDPython) [3]。

## 对Python和Coding Math的思考
多数卡尔曼过滤器教科书以及工程书籍都是由数学家或者教授写的，即便是有代码(很少)，也不能直接用于生产环境。就拿Paul Zarchan的书籍《Fundamentals of Kalman Filtering》为例，这是一本你值购买有经典的书籍，也是少有的几本提供所有实例和图表的源码的书籍，使用Fortran重新实现了卡尔曼过滤器，但是仿真和过滤器的代码混在一起，以至于很难区分他们。某些章节采用不同的方法实现同一过滤器，然后用黑体字高亮显示有修改的几行代码。

本书是关于卡尔曼过滤器的书籍，我非常关心卡尔曼过滤器的具体实现，你会发现代码实现非常简洁，数学相关的代码也很容理解，多数卡尔曼过滤器的工作其实都由数学相关的引擎或者库来完成。

可能存在的缺点就是方程的执行逻辑被隐藏了。我想让你学会如何在真实环境中使用卡尔曼过滤器，在真实项目中，你不可能在任何地方都能使用复制粘贴的方式来构建算法。

我使用Python的类来组织过滤器需要的数据，而非使用面向对象的继承等特性。比如KalmanFilter类保存了x，P，R，Q。我也看到基于面向过程的卡尔曼过滤器实现，通常都需要开发者来维护管理这些矩阵。相对于我编写的示例代码而言可能不算太糟，但是想要编写正式的卡尔曼过滤器的库，让开发人员来管理这些数据就显得不那么友好。

## 许可证 
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Kalman Filters and Random Signals in Python</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/rlabbe/Kalman-Filters-and-Random-Signals-in-Python" property="cc:attributionName" rel="cc:attributionURL">Roger Labbe</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />

http://creativecommons.org/licenses/by-nc-sa/4.0/

Based on the work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python" rel="dct:source">https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python</a>.

## 联系方式

作者：

rlabbejr@gmail.com

中文翻译：

deyuhua@gmail.com

## 参考资料

* [1] http://www.greenteapress.com/
* [2] http://nbviewer.ipython.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Chapter1.ipynb
* [3] https://github.com/barbagroup/CFDPython